## Local Inference on GPU
Model page: https://huggingface.co/microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [1]:
!pip install open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.5 MB/s eta 0:00:00


In [3]:
!pip install tqdm pandas pillow

In [4]:
import torch
import open_clip
from PIL import Image
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

device = "cuda" if torch.cuda.is_available() else "cpu"

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(
    'hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224'
)
tokenizer = open_clip.get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
model = model.to(device)
model.eval()

CustomTextCLIP(
  (visual): TimmModel(
    (trunk): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
          

In [12]:
# ----------------------------
# Load dataset
# ----------------------------
csv_path = "lesion_metadata_with_global_labels.csv"
img_dir = ""

df = pd.read_csv(csv_path)
print("Total images found:", len(df))

# Ensure matching paths
df["img_path"] = df["image_id"].apply(lambda x: os.path.join(img_dir, str(x)))

# ----------------------------
# Inference
# ----------------------------
image_embeds, text_embeds, image_ids = [], [], []

with torch.no_grad():
    for _, row in tqdm(df.iterrows(), total=len(df)):
        img_path = row["img_path"]+".jpg"
        caption = row["global_label"]

        try:
            image = preprocess_val(Image.open(img_path).convert("RGB")).unsqueeze(0).to(device)
            text = tokenizer([caption]).to(device)

            img_feat, txt_feat, _ = model(image, text)
            img_feat = img_feat / img_feat.norm(dim=-1, keepdim=True)
            txt_feat = txt_feat / txt_feat.norm(dim=-1, keepdim=True)

            image_embeds.append(img_feat.cpu().numpy())
            text_embeds.append(txt_feat.cpu().numpy())
            image_ids.append(row["image_id"])

        except Exception as e:
            print(f"Error on {row['image_id']}: {e}")


Total images found: 472


100%|██████████| 472/472 [00:16<00:00, 27.80it/s]


In [11]:
# ----------------------------
image_embeds = np.vstack(image_embeds)
text_embeds = np.vstack(text_embeds)

np.savez(
    "/content/biomedclip_global_embeddings.npz",
    image_embeds=image_embeds,
    text_embeds=text_embeds,
    image_ids=np.array(image_ids)
)

print("✅ Saved teacher embeddings at /content/biomedclip_global_embeddings.npz")

✅ Saved teacher embeddings at /content/biomedclip_global_embeddings.npz


In [17]:
import torch.nn as nn
import torch.nn.functional as F
class AttentionFusion(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.fc = nn.Linear(embed_dim * 2, 2)  # outputs 2 attention scores (img, text)

    def forward(self, img_feat, txt_feat):
        # Concatenate image and text features
        combined = torch.cat([img_feat, txt_feat], dim=-1)
        # Compute attention weights
        attn = torch.softmax(self.fc(combined), dim=-1)
        # Weighted fusion
        fused = attn[:, 0:1] * img_feat + attn[:, 1:2] * txt_feat
        # Normalize final embedding
        fused = F.normalize(fused, dim=-1)
        return fused, attn

# Initialize attention fusion
fusion_layer = AttentionFusion(embed_dim=512).to(device)
fusion_layer.eval()  # we're not training yet, just generating teacher embeddings

# ----------------------------
# 3️⃣ Load Dataset (CSV)
# ----------------------------
csv_path = "lesion_metadata_with_global_labels.csv"
img_dir = ""

df = pd.read_csv(csv_path)
print("✅ Total samples:", len(df))

df["img_path"] = df["image_id"].apply(lambda x: os.path.join(img_dir, str(x)))
df["img_path"]+=".jpg"

# ----------------------------
# 4️⃣ Generate Attention-Fused Embeddings
# ----------------------------
fused_embeds, attn_weights, image_ids = [], [], []

with torch.no_grad():
    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            image = preprocess_val(Image.open(row["img_path"]).convert("RGB")).unsqueeze(0).to(device)
            text = tokenizer([row["global_label"]]).to(device)

            img_feat, txt_feat, _ = model(image, text)
            img_feat = F.normalize(img_feat, dim=-1)
            txt_feat = F.normalize(txt_feat, dim=-1)

            fused_feat, attn = fusion_layer(img_feat, txt_feat)

            fused_embeds.append(fused_feat.cpu().numpy())
            attn_weights.append(attn.cpu().numpy())
            image_ids.append(row["image_id"])

        except Exception as e:
            print(f"⚠️ Error on {row['image_id']}: {e}")

✅ Total samples: 472


100%|██████████| 472/472 [00:17<00:00, 27.25it/s]


In [18]:
# ----------------------------
fused_embeds = np.vstack(fused_embeds)
attn_weights = np.vstack(attn_weights)

np.savez(
    "/content/biomedclip_global_fused_attention_embeddings.npz",
    fused_embeds=fused_embeds,
    attn_weights=attn_weights,
    image_ids=np.array(image_ids)
)

print("🎯 Saved fused teacher embeddings with attention at /content/biomedclip_global_fused_attention_embeddings.npz")

🎯 Saved fused teacher embeddings with attention at /content/biomedclip_global_fused_attention_embeddings.npz
